# 청와대 국민청원 데이터
## 청원 분류 예측하기
* 데이터 출처 : https://github.com/akngs/petitions
* 일부 기간의 청원을 통해 모델을 만들고 일부 기간의 청원 분류를 임의로 예측해 보고 실제 분류와의 차이를 알아본다.

In [1]:
# 출력데이터가 지저분하게 보이지 않도록 warnings을 불러왔다.
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from plotnine import *
print(pd.__version__)
print(np.__version__)

0.21.0
1.14.0


In [3]:
petitions = pd.read_csv('data/petition.csv', index_col=0, parse_dates=['start', 'end'])
# 데이터의 크기가 어느정도인지 본다.
petitions.shape

(192387, 7)

In [4]:
petitions.head()

,start,end,answered,votes,category,title,content
article_id,,,,,,,
21,2017-08-19,2017-11-17,0,9,안전/환경,스텔라 데이지호에 대한 제안입니다.,스텔라 데이지호에 대한 제안입니다.\n3월31일 스텔라 데이지호가 침몰하고 5달째가...
22,2017-08-19,2017-11-17,0,17,기타,비리제보처를 만들어주세요.,현 정부에 국민들이 가장 원하는 것은 부패척결입니다. 우리 사회에 각종 비리들이 ...
23,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ..."
24,2017-08-19,2017-08-26,0,53,일자리,공공기관 무조건적인 정규직전환을 반대합니다.,현정부에서 정규직 일자리를 늘리는 것에 찬성합니다. 그런데 공공기관 비정규직들은 인...
25,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ..."


In [5]:
petitions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192387 entries, 21 to 242902
Data columns (total 7 columns):
start       192387 non-null datetime64[ns]
end         192387 non-null datetime64[ns]
answered    192387 non-null int64
votes       192387 non-null int64
category    192387 non-null object
title       192387 non-null object
content     192386 non-null object
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 11.7+ MB


## 학습데이터와 테스트 데이터를 만든다.

In [6]:
train = petitions.loc[(petitions['start'] >= '2018-05-01') & (petitions['start'] < '2018-05-15')]
train.shape

(10984, 7)

In [7]:
test = petitions.loc[(petitions['start'] >= '2018-05-15') & (petitions['start'] < '2018-05-31')]
test.shape

(7067, 7)